In [2]:
import numpy as np
import pandas as pd

### Read and display info about our data

In [3]:
df = pd.read_csv('my-users.csv')
df

,username,city,smoker,income,experience,fav_food
0,Eva,Sofia,No,1250.00,1.3,salad
1,Iva,Plovdiv,No,2345.00,2.0,salad
2,Georgi,Montana,No,1987.50,3.8,pizza
3,Toni,Sofia,Yes,2500.90,4.0,salad
4,Misho,Burgas,No,4500.50,5.0,soup
5,Biser,Sofia,No,5555.55,9.2,salad
6,Evgeni,Montana,Yes,3400.00,5.8,salad
7,Galq,Sofia,Yes,8000.90,3.0,pizza
8,Tanq,Ruse,No,3560.00,2.4,salad
9,Tedi,Sofia,Yes,3560.00,8.0,potatoes


In [4]:
df.columns

Index(['username', 'city', 'smoker', 'income', 'experience', 'fav_food'], dtype='object')

In [5]:
df.index

RangeIndex(start=0, stop=15, step=1)

In [6]:
df.describe()

,income,experience
count,15.000000,15.000000
mean,3680.122000,4.293333
std,1788.632646,2.247051
min,1250.000000,1.300000
25%,2345.000000,2.450000
50%,3560.000000,4.000000
75%,4675.305000,5.400000
max,8000.900000,9.200000


### Data manipulation

In [7]:
bonus = np.round(df['income'] * df['experience'] / 100, 2)
df['bonus'] = bonus
bonus

0      16.25
1      46.90
2      75.53
3     100.04
4     225.02
5     511.11
6     197.20
7     240.03
8      85.44
9     284.80
10     56.28
11    194.50
12     47.52
13    194.00
14    333.33
dtype: float64

In [8]:
# get a row based on index
df.iloc[0]

username         Eva
city           Sofia
smoker            No
income        1250.0
experience       1.3
fav_food       salad
bonus          16.25
Name: 0, dtype: object

In [9]:
# conditional filtering
df[(df['income'] > 3000) & (df['city'] == 'Sofia')]

,username,city,smoker,income,experience,fav_food,bonus
5,Biser,Sofia,No,5555.55,9.2,salad,511.11
7,Galq,Sofia,Yes,8000.90,3.0,pizza,240.03
9,Tedi,Sofia,Yes,3560.00,8.0,potatoes,284.80
14,Zoq,Sofia,No,5555.55,6.0,salad,333.33


In [10]:
def mask_fav_food(food):
    if len(food) < 3:
        return food
    else:
        middle_letters = food[1:-1]
        asterisks = '*' * len(middle_letters)
        return food[0] + asterisks + food[-1]

df['masked_food'] = df['fav_food'].apply(mask_fav_food)
df.head()

,username,city,smoker,income,experience,fav_food,bonus,masked_food
0,Eva,Sofia,No,1250.0,1.3,salad,16.25,s***d
1,Iva,Plovdiv,No,2345.0,2.0,salad,46.90,s***d
2,Georgi,Montana,No,1987.5,3.8,pizza,75.53,p***a
3,Toni,Sofia,Yes,2500.9,4.0,salad,100.04,s***d
4,Misho,Burgas,No,4500.5,5.0,soup,225.02,s**p


In [11]:
def action_type(income, experience):
    if income < 3000 and experience > 3:
        return "Promotion required"
    else:
        return "Other"

# apply method on more than one row
# df['action'] = df[['income','experience']].apply(lambda df: action_type(df['income'], df['experience']),axis=1)
df['action'] = np.vectorize(action_type)(df['income'], df['experience'])
df.head()

,username,city,smoker,income,experience,fav_food,bonus,masked_food,action
0,Eva,Sofia,No,1250.0,1.3,salad,16.25,s***d,Other
1,Iva,Plovdiv,No,2345.0,2.0,salad,46.90,s***d,Other
2,Georgi,Montana,No,1987.5,3.8,pizza,75.53,p***a,Promotion required
3,Toni,Sofia,Yes,2500.9,4.0,salad,100.04,s***d,Promotion required
4,Misho,Burgas,No,4500.5,5.0,soup,225.02,s**p,Other


In [12]:
df['income'].max()

8000.9

In [13]:
df['city'].value_counts()

Sofia      7
Montana    3
Ruse       2
Plovdiv    1
Burgas     1
Shumen     1
Name: city, dtype: int64

### Grouping data

In [14]:
# we specify which columns to be returned, because not all columns are of number value and we cannot calculate mean for all columns
df.groupby('city')[['income', 'experience']].mean()

,income,experience
city,,
Burgas,4500.500000,5.000000
Montana,2577.500000,4.000000
Plovdiv,2345.000000,2.000000
Ruse,3725.000000,3.700000
Shumen,4850.110000,4.000000
Sofia,4046.245714,4.857143


In [15]:
double_group = df.groupby(['city', 'smoker'])[['income', 'experience']].mean()
print(double_group.loc[('Sofia', 'No')])
double_group

income        4120.366667
experience       5.500000
Name: (Sofia, No), dtype: float64


income  experience
city    smoker                         
Burgas  No      4500.500000       5.000
Montana No      2166.250000       3.100
        Yes     3400.000000       5.800
Plovdiv No      2345.000000       2.000
Ruse    No      3725.000000       3.700
Shumen  No      4850.110000       4.000
Sofia   No      4120.366667       5.500
        Yes     3990.655000       4.375

### Concatenation

In [16]:
data_one = {'A': ['A0', 'A1', 'A2', 'A3'],'B': ['B0', 'B1', 'B2', 'B3']}
data_two = {'C': ['C0', 'C1', 'C2', 'C3'], 'D': ['D0', 'D1', 'D2', 'D3']}

one = pd.DataFrame(data_one)
two = pd.DataFrame(data_two)

axis1 = pd.concat([one, two], axis=1)
axis1

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


### Merge

In [17]:
people_one = pd.DataFrame({'in_id':[1,2,3,4],'name':['Ani','Biser','Valeri','Georgi']})
people_two = pd.DataFrame({'out_id':[1,2,3,4],'name':['Jana','Ani','Ioana','Biser']})

pd.merge(people_one, people_two, how='inner', on='name')

,in_id,name,out_id
0,1,Ani,2
1,2,Biser,4
